In [1]:
!pip install unsloth==2025.9.7 unsloth_zoo==2025.9.9 numpy==2.2.6 matplotlib==3.10.6 scikit-learn==1.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 10.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 10.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 182.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 188.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 199.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49

In [2]:
!pip install --upgrade --no-deps --no-build-isolation flash-attn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 81.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp311-cp311-linux_x86_64.whl size=116548888 sha256=2796491b466b0d1924beeca8b5a16891016187111557f7dd003d55267f75627e
  Stored in directory: /home/runai-home/.cache/pip/wheels/42/31/1f/4b22dd7295b3cb064b8fa9038f6d58fb15c9571555b2d7c39c
Successfully built flash-attn


In [3]:
!python --version

Python 3.11.13


In [4]:
import torch
torch.__version__

'2.9.1+cu128'

In [5]:
import flash_attn
flash_attn.__version__

'2.8.3'

In [6]:
patch = b'@@ -272,7 +277,7 @@\r\n \r\n         # Mistral Nemo 12b has weird dimensions\r\n         if attention_size != hidden_size:\r\n-            self.temp_O = torch.empty((1, bsz, hidden_size), dtype = dtype, device = device)\r\n+            self.temp_O = torch.empty((bsz, 1, hidden_size), dtype = dtype, device = device)\r\n         else:\r\n             self.temp_O = self.temp_QA[1][:,:,:hidden_size]\r\n         pass\r\n@@ -333,52 +338,12 @@\r\n     Kn = self.paged_attention_K[:kv_seq_len].permute(1, 2, 0, 3)\r\n     Vn = self.paged_attention_V[:kv_seq_len].permute(1, 2, 0, 3)\r\n \r\n-    # Handle sliding windows\r\n-    sliding_window = getattr(self.config, "sliding_window", None)\r\n-    if sliding_window is not None and kv_seq_len > sliding_window:\r\n-        # From https://github.com/huggingface/transformers/blob/main/src/transformers/models/mistral/modeling_mistral.py#L193\r\n-        slicing_tokens = 1 - sliding_window\r\n-        Knn = Kn[:, :, slicing_tokens:, :]#.contiguous()\r\n-        Vnn = Vn[:, :, slicing_tokens:, :]#.contiguous()\r\n-    else:\r\n-        Knn, Vnn = Kn, Vn\r\n-    pass\r\n+    Qnn = Qn.transpose(1, 2)\r\n+    Knn = Kn.transpose(1, 2)\r\n+    Vnn = Vn.transpose(1, 2)\r\n \r\n-    # when qlen==vlen and attn_mask is None, we should use causal attention\r\n-    Q_len = Qn.shape[-2]\r\n-    K_len = Knn.shape[-2]\r\n-    if attention_mask is None and Q_len == K_len:\r\n-        is_causal = True\r\n-    else:\r\n-        is_causal = False\r\n+    A = flash_attn_func(Qnn, Knn, Vnn)\r\n \r\n-    # Grouped query attention\r\n-    _, _, cached_len, _ = Knn.shape\r\n-    if bsz == 1 or not SDPA_HAS_GQA and n_groups != 1:\r\n-        Knn = Knn[:, :, None, :, :].expand(bsz, n_kv_heads, n_groups, cached_len, head_dim)\r\n-        Vnn = Vnn[:, :, None, :, :].expand(bsz, n_kv_heads, n_groups, cached_len, head_dim)\r\n-        Knn = Knn.reshape(bsz, n_heads, cached_len, head_dim)\r\n-        Vnn = Vnn.reshape(bsz, n_heads, cached_len, head_dim)\r\n-    pass\r\n-    # else:\r\n-    #     Knn, Vnn = Knn, Vnn\r\n-    # pass\r\n-\r\n-    # Attention\r\n-    if bsz == 1:\r\n-        Qn *= self.scalar # See https://github.com/ggerganov/llama.cpp/issues/7805#issuecomment-2153349963\r\n-        # It seems like doing (Q * scalar) @ K is better than (Q @ K) * scalar to stop overflows\r\n-        A = torch_matmul(Qn, Knn.transpose(2, 3), out = self.attention[:,:,:,:cached_len])\r\n-        # if attention_mask is not None: A += attention_mask # Must add attention_mask for batched\r\n-        A[:] = torch_nn_functional_softmax(A, dim = -1, dtype = torch.float32)#.to(A.dtype)\r\n-        A = torch_matmul(A, Vnn, out = Qn)\r\n-    else:\r\n-        if SDPA_HAS_GQA:\r\n-            A = scaled_dot_product_attention(Qn, Knn, Vnn, attn_mask = attention_mask, is_causal = is_causal, enable_gqa = True)\r\n-        else:\r\n-            A = scaled_dot_product_attention(Qn, Knn, Vnn, attn_mask = attention_mask, is_causal = is_causal)\r\n-    pass\r\n-    A = A.transpose(1, 2)\r\n     A = A.reshape(bsz, 1, attention_size)\r\n     A = fast_linear_forward(self.o_proj, A, out = self.temp_O)\r\n     return A, (Kn, Vn)\r\n'
with open("qwen3.patch", "wb") as f:
    f.write(patch)
print(patch.decode())

@@ -272,7 +277,7 @@
 
         # Mistral Nemo 12b has weird dimensions
         if attention_size != hidden_size:
-            self.temp_O = torch.empty((1, bsz, hidden_size), dtype = dtype, device = device)
+            self.temp_O = torch.empty((bsz, 1, hidden_size), dtype = dtype, device = device)
         else:
             self.temp_O = self.temp_QA[1][:,:,:hidden_size]
         pass
@@ -333,52 +338,12 @@
     Kn = self.paged_attention_K[:kv_seq_len].permute(1, 2, 0, 3)
     Vn = self.paged_attention_V[:kv_seq_len].permute(1, 2, 0, 3)
 
-    # Handle sliding windows
-    sliding_window = getattr(self.config, "sliding_window", None)
-    if sliding_window is not None and kv_seq_len > sliding_window:
-        # From https://github.com/huggingface/transformers/blob/main/src/transformers/models/mistral/modeling_mistral.py#L193
-        slicing_tokens = 1 - sliding_window
-        Knn = Kn[:, :, slicing_tokens:, :]#.contiguous()
-        Vnn = Vn[:, :, slicing_tokens:, :]#.contiguous()

In [7]:
!patch --binary /usr/local/lib/python3.11/dist-packages/unsloth/models/qwen3.py qwen3.patch

patching file /usr/local/lib/python3.11/dist-packages/unsloth/models/qwen3.py
